## ToG with link predictor and tail selector

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import torch
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from utils.graph import KGraphPreproc
from utils.graph.tog_lp import ToGLPChain
from utils.llm.mistral import MistralLLM
from utils.llm.qwen import Qwen2_5
from utils.file import export_results_to_file
import ast
import os
os.environ["HF_HOME"] = "/models"
os.environ["TRANSFORMERS_CACHE"] = "/models"
device = torch.device("cuda:1")
torch.cuda.set_device(device)
torch.set_default_device(device)

### ToG Algorithm pseudocode
given a list of entities $E^{D-1}$:
1. Select all relations $R^D$
2. Select top-N relevant $E^{D-1}$-$R^D$-? | ?-$R^D$-$E^{D-1}$
3. Extend P by top-N paths
4. Decide if enough to answer the question (YES/NO)
5. Repeat (NO), return (YES)


### 0. Init

### 2. Select top relevant candidate triplets 

## QA Tests

In [4]:
qwen = Qwen2_5()

In [5]:
mistral = MistralLLM()

In [7]:
sbert = SentenceTransformer("all-mpnet-base-v2")
sbert_cache_path = "/datasets/FB15k-237/cache/sbert.csv"

In [8]:
fbkb_graph = KGraphPreproc.get_fbkb_graph()

fbkb_graph.embed_triplets(
    embedding_function=lambda q: sbert.encode(q),
    cache_path=sbert_cache_path
)

Checking embedding cache
Loading embedding cache


329136it [11:43, 468.10it/s]


In [22]:
def extract_cwq_topic_entities(row):
    topic_ids = ast.literal_eval(row["topic_ids"])
    topic_entities = []
    for topic_id in topic_ids:
        if topic_id in fbkb_graph.mid2name:
            topic_entities.append(fbkb_graph.mid2name[topic_id])
    return topic_entities

In [11]:
chain = ToGLPChain.from_llm(
    llm=mistral,
    link_predictor_llm=qwen,
    graph=fbkb_graph,
    sbert=sbert,
    verbose=False
)

/model/utils/graph/tog_lp.py:236: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(llm=llm, prompt=qa_prompt)


In [12]:
cwq = pd.read_csv("/datasets/CWQ/cwq-1000.csv", index_col=0)

In [13]:
experiment_name = f"tog-lp-4"
res_path = f"/datasets/CWQ/results/{experiment_name}.csv"

In [ ]:
results = []
id_list = []
l = 0
if os.path.isfile(res_path):
    r_df = pd.read_csv(res_path)
    l = len(r_df)
    results = list(r_df.Model.values)
for c, (i, r) in enumerate(tqdm(list(cwq.iterrows()))):
    id_list.append(i)
    if c < l:
            continue
    q = r.question
    topic_entities = extract_cwq_topic_entities(r)
    response = chain.invoke(
        input={
            "query": q,
            "topic_entities": topic_entities,
    })
    answer = response["result"]["text"]
    depth = response["depth"]
    results.append((answer, depth))
    if c % 10 == 0:
        export_results_to_file(res_path, results, id_list)
export_results_to_file(res_path, results, id_list)


100%|██████████| 1000/1000 [16:05:23<00:00, 57.92s/it]   


/model/utils/graph/tog_lp.py:128: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = self.evaluate_context_chain(
 48%|████▊     | 485/1000 [8:27:18<11:45:30, 82.20s/it] 